In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
chosen_ramris:str = 'D:\\ESMIRA\\SPSS data\\5. CSA_T1_MRI_scores_SPSS.csv'
df = pd.read_csv(chosen_ramris, sep=';')
WRs = [column for column in df if column[:2]=='WR']
WRs.append('CSANUMM')
dfwr = df[WRs]  # df of wrist + csaid
# 此时获得了所有的WR的列

In [ ]:
# 需要计算所有Wrist的TSY, SYN, BME. 合并.1和.2
TSY = [column for column in dfwr if (column[2:5]=='TSY' or column=='CSANUMM')]
SYN = [column for column in dfwr if (column[2:5]=='SYN' or column=='CSANUMM')]
BME = [column for column in dfwr if (column[2:5]=='BME' or column=='CSANUMM')]
dfwr_tsy = df[TSY]
dfwr_syn = df[SYN]
dfwr_bme = df[BME]
# 此时获得了各类的列

In [ ]:
def meanreader(df:pd.DataFrame):
    meancolumn = [column[:-2] for column in df if column[-2:]=='.1']
    for col in meancolumn:
        col1, col2 = col+'.1', col+'.2'
        df.loc[:, col1] = pd.to_numeric(df[col1], errors='coerce')
        df.loc[df[col1]>10, col1] = 0
        df.loc[:, col2] = pd.to_numeric(df[col2], errors='coerce')
        df.loc[df[col2]>10, col2] = 0
        df.loc[:, col] = df[[col1, col2]].mean(axis=1, skipna=True)
    leftcolumn = ['CSANUMM']
    leftcolumn.extend(meancolumn)
    return df[leftcolumn]
meantsy = meanreader(dfwr_tsy.copy())
meansyn = meanreader(dfwr_syn.copy())
meanbme = meanreader(dfwr_bme.copy())
# 计算两个rater的平均值

In [ ]:
def sumscore(df:pd.DataFrame) ->pd.DataFrame:
    scorecolumn = [column for column in df if column!='CSANUMM']
    df['sums'] = df[scorecolumn].sum(axis=1, skipna=True)
    return df[['CSANUMM', 'sums']]
sumtsy:pd.DataFrame = sumscore(meantsy)  # CSANUMM, sumtsy
sumsyn:pd.DataFrame = sumscore(meansyn)
sumbme:pd.DataFrame = sumscore(meanbme)
# 计算该类炎症的总和值

In [ ]:
# 获得预测值-以计算预测误差
def getdiff(feature):
    chosen_ramris:str = f'D:\\ESMIRAcode\\RAMRISinfer\\output\\RAMRISestimation_sum_CSA\\Wrist_{feature}_CSA_sumTrue.csv'
    df = pd.read_csv(chosen_ramris, sep=';')
    df['diff'] = df['sums'] - df['sums_gt']
    df['CSANUMM'] = df['ID'].str.slice(3).astype(int)
    return df[['CSANUMM', 'diff']]
# 获得CSANUMM和Diff
difftsy:pd.DataFrame = getdiff('TSY')
diffsyn:pd.DataFrame = getdiff('SYN')
diffbme:pd.DataFrame = getdiff('BME')

In [ ]:
#合并两个列表，使得其同时有sum和diff
ultitsy = pd.merge(sumtsy, difftsy, how='left', on='CSANUMM')
ultisyn = pd.merge(sumsyn, diffsyn, how='left', on='CSANUMM')
ultibme = pd.merge(sumbme, diffbme, how='left', on='CSANUMM')
# CSANUMM, sums, diff

In [ ]:
# 看一眼分布情况才能确定后面的bin和bin2
# 看bin
tsycount = sumtsy['sums'].value_counts()
syncount = sumsyn['sums'].value_counts()
bmecount = sumbme['sums'].value_counts()

fig, axs = plt.subplots(1, 3, figsize=(12, 5))
bars1 = axs[0].bar(tsycount.index, tsycount.values)
axs[0].set_title('TSY')
axs[0].set_ylabel('Values')
# 在第一个子图上显示每个柱的值
for bar in bars1:
    yval = bar.get_height()
    axs[0].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

bars2 = axs[1].bar(syncount.index, syncount.values)
axs[1].set_title('SYN')
axs[1].set_ylabel('Values')
# 在第二个子图上显示每个柱的值
for bar in bars2:
    yval = bar.get_height()
    axs[1].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

bars3 = axs[2].bar(bmecount.index, bmecount.values)
axs[2].set_title('BME')
axs[2].set_ylabel('Values')
# 在第三个子图上显示每个柱的值
for bar in bars3:
    yval = bar.get_height()
    axs[2].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')


plt.tight_layout()
plt.show()

In [ ]:
# 看bin2
tsycount = sumtsy['diff'].value_counts()
syncount = sumsyn['diff'].value_counts()
bmecount = sumbme['diff'].value_counts()

fig, axs = plt.subplots(1, 3, figsize=(12, 5))
bars1 = axs[0].bar(tsycount.index, tsycount.values)
axs[0].set_title('TSY')
axs[0].set_ylabel('Values')
# 在第一个子图上显示每个柱的值
for bar in bars1:
    yval = bar.get_height()
    axs[0].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

bars2 = axs[1].bar(syncount.index, syncount.values)
axs[1].set_title('SYN')
axs[1].set_ylabel('Values')
# 在第二个子图上显示每个柱的值
for bar in bars2:
    yval = bar.get_height()
    axs[1].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')

bars3 = axs[2].bar(bmecount.index, bmecount.values)
axs[2].set_title('BME')
axs[2].set_ylabel('Values')
# 在第三个子图上显示每个柱的值
for bar in bars3:
    yval = bar.get_height()
    axs[2].text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha='center', va='bottom')


plt.tight_layout()
plt.show()

In [ ]:
# 根据sums和diff的值分别进行分组
def getsample(df:pd.DataFrame, bin:list, bin2:list, numsample:int=2):
    # 定义函数，根据分数将数据分组，每2分为一个组
    df['group1'] = pd.cut(df['sums'], bins=bin, right=False)
    df['group2'] = pd.cut(df['diff'], bins=bin2, right=False)
    # 对每个分数区间分组，并在每个组中采样 2 个样本
    sampled_df = df.groupby(['group1', 'group2'], group_keys=False).apply(lambda x: x.sample(numsample, random_state=42))
    # 删除 group 分组后的多余层
    sampled_df = sampled_df.reset_index(drop=True)
    return sampled_df
sampletsy = getsample(ultitsy, bin=[0, 1, 2, 4, 12], bin2=[], numsample=12)
samplesyn = getsample(ultisyn, bin=[0, 1, 2, 6], bin2=[],umsample=15)
samplebme = getsample(ultibme, bin=[0, 1, 2, 3, 15], bin2=[], numsample=12)

In [ ]:
# 以下为三个加一起来看的办法 -- 因为没法同时满足个体和群体都是均匀分布

In [ ]:
# 合并三个一起
def merge(df1:pd.DataFrame, df2:pd.DataFrame, df3:pd.DataFrame):
    res = df1.merge(df2, on='CSANUMM', how='inner').merge(df3, on='CSANUMM', how='inner')  # tsy+syn+bme
    return res
res = merge(sumtsy, sumsyn, sumbme)
print(res.head(10))
print(f'merged: {res.shape}, tsy: {sumtsy.shape}, syn: {sumsyn.shape}, bme: {sumbme.shape}')

# 看合并之后的bin
def sumreader2(df:pd.DataFrame) ->pd.DataFrame:
    scorecolumn = [column for column in df if 'sums' in column]
    diffcolumn = [column for column in df if 'diff' in column]
    df['allsum'] = df[scorecolumn].sum(axis=1, skipna=True)
    df['alldiff'] = df[diffcolumn].sum(axis=1, skipna=True)
    return df[['CSANUMM', 'allsum', 'sums_x', 'sums_y', 'sums', 'alldiff', 'diff_x', 'diff_y', 'diff']]

res = sumreader2(res)
print(res.head(10))
plt.hist(res.allsum.values, bins=30)
plt.show()

In [ ]:
def getsample2(df:pd.DataFrame, bin:list, bin2:list, numsample:int=2):
    # 定义函数，根据分数将数据分组，每2分为一个组
    df['group1'] = pd.cut(df['allsum'], bins=bin, right=False)
    df['group2'] = pd.cut(df['allsum'], bins=bin2, right=False)
    # print(df.head(20))
    # 对每个分数区间分组，并在每个组中采样 2 个样本
    sampled_df = df.groupby(['group1', 'group2'], group_keys=False).apply(lambda x: x.sample(numsample, random_state=42))
    # 删除 group 分组后的多余层
    sampled_df = sampled_df.reset_index(drop=True)
    return sampled_df

item = getsample2(res, bin=[0, 1, 2, 3, 4, 6, 8, 12, 35], bin2=, numsample=15)
print(item.shape)
# print(item.sort_values(by='allsum', ascending=True).head(20))
print(item.CSANUMM.to_list())